In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

base_folder_lt3 = r'D:\measuring\data\Purification_lt3_raw_data'
base_folder_lt4 = r'D:\measuring\data\Purification_lt4_raw_data'

### ssro calibrations
day = '20160716_' # for ssro calibration
ssro_calib_lt3 = day+'091002'
ssro_calib_lt4 = day+'113952'

thetas = [np.pi/4,np.pi/5,np.pi/6,np.pi/8]
addenda = [r'\Purify_XX_First_Attempts',r'\Purify_35percent_theta',r'\Purify_25percent_theta',r'\Purify_15percent_theta']

outcome_bins = np.arange(50,540,50)

In [ ]:
def calculate_ebit_rate(outcome_bins,apply_ROC = True):
    
    ## get correlation probabilties
    psi_m_XX,psi_p_XX,psi_m_XX_corrs,psi_p_XX_corrs = Purify_XX.sweep_filter_parameter_vs_correlations('bin_w2',outcome_bins,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_YY,psi_p_YY,psi_m_YY_corrs,psi_p_YY_corrs = Purify_YY.sweep_filter_parameter_vs_correlations('bin_w2',outcome_bins,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_ZZ,psi_p_ZZ,psi_m_ZZ_corrs,psi_p_ZZ_corrs = Purify_ZZ.sweep_filter_parameter_vs_correlations('bin_w2',outcome_bins,
                                                                         apply_ROC = apply_ROC,do_plot= False)       
    ## calculate fidelities
    psi_m_F = (psi_m_XX[1] + psi_m_YY[1] + psi_m_ZZ[1]+1)/4.
    psi_p_F = (psi_p_XX[1] + psi_p_YY[1] + psi_p_ZZ[1]+1)/4.
    
    ## calciulate error bars
    psi_m_F_u = np.sqrt(psi_m_XX[2]**2 + psi_m_YY[2]**2 + psi_m_ZZ[2]**2)/4.
    psi_p_F_u = np.sqrt(psi_p_XX[2]**2 + psi_p_YY[2]**2 + psi_p_ZZ[2]**2)/4.
    
    # calculate ebits    
    ebits_p, ebits_p_u = pa.calculate_ebits(psi_m_YY,psi_m_ZZ,psi_m_XX_corrs)
    ebits_m, ebits_m_u = pa.calculate_ebits(psi_p_YY,psi_p_ZZ,psi_p_XX_corrs)
    
    total_counts_p = psi_p_XX[0]+psi_p_YY[0]+psi_p_ZZ[0]
    total_counts_m = psi_m_XX[0]+psi_m_YY[0]+psi_m_ZZ[0]
    
    total_time = Purify_XX.estimate_sequence_time() + Purify_YY.estimate_sequence_time() + Purify_ZZ.estimate_sequence_time()
    
    rate_p = np.array(total_counts_p)/total_time
    rate_m = np.array(total_counts_m)/total_time
    
    ebit_rate_p = np.sum(rate_p*ebits_p)
    ebit_rate_p_u = np.sqrt(np.sum(rate_p**2*ebits_p_u**2))
    ebit_rate_m = np.sum(rate_m*ebits_m)
    ebit_rate_m_u = np.sqrt(np.sum(rate_m**2*ebits_m_u**2))
    
    avg_fidelity_p = np.sum(psi_p_F*total_counts_p)/np.sum(total_counts_p)
    avg_fidelity_p_u = np.sqrt(np.sum(psi_p_F_u**2*total_counts_p**2))/np.sum(total_counts_p)
    avg_fidelity_m = np.sum(psi_m_F*total_counts_p)/np.sum(total_counts_m)
    avg_fidelity_m_u = np.sqrt(np.sum(psi_m_F_u**2*total_counts_p**2))/np.sum(total_counts_m)
    
    rate_p = np.sum(rate_p)
    rate_m = np.sum(rate_m)
    
    return [rate_p,rate_m,ebit_rate_p,ebit_rate_p_u,ebit_rate_m,ebit_rate_m_u,avg_fidelity_p,avg_fidelity_p_u,avg_fidelity_m,avg_fidelity_m_u]

In [ ]:
vals = []

for i,theta in enumerate(thetas):

    folder_lt3 = base_folder_lt3+addenda[i]
    folder_lt4 = base_folder_lt4+addenda[i]
    
    Purify_XX = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_YY = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_ZZ = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

    if theta == np.pi/6:
        ### find timestamps for all three experiments (for pi/6)
        ZZ_days = ['20160808','20160807','20160722','20160721','20160720']
        XX_days = ZZ_days
        YY_days = ['20160808','20160807','20160722','20160724','20160725']
        ZZ_days_new_timings, XX_days_new_timings, YY_days_new_timings = ['20160811'],['20160811'],['20160811']

    elif theta == np.pi/4:
        ZZ_days = ['20160714','20160715','20160716','20160718','20160719']
        XX_days = ZZ_days
        YY_days = ['20160727','20160726']
        ZZ_days_new_timings, XX_days_new_timings, YY_days_new_timings = ['20160816'],['20160816'],['20160816']

    elif theta == np.pi/5:
        ZZ_days_new_timings = ['20160818','20160819']; YY_days_new_timings = ZZ_days_new_timings; XX_days_new_timings = ZZ_days_new_timings
        ZZ_days = []; YY_days = []; XX_days = []


    elif theta == np.pi/8:    
        ZZ_days_new_timings = ['20160820','20160821','20160822']; YY_days_new_timings = ZZ_days_new_timings; XX_days_new_timings = ZZ_days_new_timings
        ZZ_days = []; YY_days = []; XX_days = []

    all_lt3_ZZ , all_lt4_ZZ = [],[]
    all_lt3_XX , all_lt4_XX = [],[]
    all_lt3_YY , all_lt4_YY = [],[]

    offsets_XX,offsets_YY,offsets_ZZ = [],[],[] # Hold offset to compensate for timing change for new APD
    correction_time = 2772.5e3-2791.5e3
    offsets_ch1_XX,offsets_ch1_YY,offsets_ch1_ZZ = [],[],[]
    start_offset_ch1 = 1.8e3

    for d in ZZ_days:

        tstamp_lt3,tstamp_lt4 = Purify_ZZ.tstamps_for_both_setups(d,contains = 'ZZ')
        #,newest_tstamp = '110000') ### newest timestamp allows for only taking parts of a day.
        all_lt3_ZZ.extend(tstamp_lt3)
        all_lt4_ZZ.extend(tstamp_lt4)
        offsets_ZZ.extend(np.zeros(np.shape(tstamp_lt3)))
        offsets_ch1_ZZ.extend(np.zeros(np.shape(tstamp_lt4)))

    for d in YY_days:

        tstamp_lt3,tstamp_lt4 = Purify_YY.tstamps_for_both_setups(d,contains = 'YY')
        all_lt3_YY.extend(tstamp_lt3)
        all_lt4_YY.extend(tstamp_lt4)
        offsets_YY.extend(np.zeros(np.shape(tstamp_lt3)))
        offsets_ch1_YY.extend(np.zeros(np.shape(tstamp_lt4)))

    for d in XX_days:

        tstamp_lt3,tstamp_lt4 = Purify_XX.tstamps_for_both_setups(d,contains = 'XX')
        all_lt3_XX.extend(tstamp_lt3)
        all_lt4_XX.extend(tstamp_lt4)
        offsets_XX.extend(np.zeros(np.shape(tstamp_lt3)))
        offsets_ch1_XX.extend(np.zeros(np.shape(tstamp_lt4)))

    for d in ZZ_days_new_timings:

        tstamp_lt3,tstamp_lt4 = Purify_ZZ.tstamps_for_both_setups(d,contains = 'ZZ')
        #,newest_tstamp = '110000') ### newest timestamp allows for only taking parts of a day.
        all_lt3_ZZ.extend(tstamp_lt3)
        all_lt4_ZZ.extend(tstamp_lt4)
        offsets_ZZ.extend(np.zeros(np.shape(tstamp_lt3))+ correction_time)
        offsets_ch1_ZZ.extend(np.zeros(np.shape(tstamp_lt4))+ start_offset_ch1)

    for d in YY_days_new_timings:

        tstamp_lt3,tstamp_lt4 = Purify_YY.tstamps_for_both_setups(d,contains = 'YY')
        all_lt3_YY.extend(tstamp_lt3)
        all_lt4_YY.extend(tstamp_lt4)
        offsets_YY.extend(np.zeros(np.shape(tstamp_lt3))+ correction_time)
        offsets_ch1_YY.extend(np.zeros(np.shape(tstamp_lt4))+ start_offset_ch1)

    for d in XX_days_new_timings:

        tstamp_lt3,tstamp_lt4 = Purify_XX.tstamps_for_both_setups(d,contains = 'XX')
        all_lt3_XX.extend(tstamp_lt3)
        all_lt4_XX.extend(tstamp_lt4)
        offsets_XX.extend(np.zeros(np.shape(tstamp_lt3))+ correction_time)
        offsets_ch1_XX.extend(np.zeros(np.shape(tstamp_lt4))+ start_offset_ch1) 
        
    ### load the data
    Purify_XX.load_raw_data(all_lt3_XX,all_lt4_XX)
    Purify_XX.correct_pq_times(offsets = offsets_XX, offsets_ch1 = offsets_ch1_XX)
    Purify_YY.load_raw_data(all_lt3_YY,all_lt4_YY)
    Purify_YY.correct_pq_times(offsets = offsets_YY, offsets_ch1 = offsets_ch1_YY)
    Purify_ZZ.load_raw_data(all_lt3_ZZ,all_lt4_ZZ)
    Purify_ZZ.correct_pq_times(offsets = offsets_ZZ, offsets_ch1 = offsets_ch1_ZZ)
    
    vals.append(calculate_ebit_rate(outcome_bins = outcome_bins))
   
vals = np.transpose(vals)

In [ ]:
rate_p = vals[0]
rate_m = vals[1]
ebit_rate_p = vals[2]
ebit_rate_p_u = vals[3]
ebit_rate_m = vals[4]
ebit_rate_m_u = vals[5]
avg_fidelity_p = vals[6]
avg_fidelity_p_u = vals[7]
avg_fidelity_m = vals[8]
avg_fidelity_m_u = vals[9]

super_posn_frac = np.sin(thetas)**2

fig  = plt.figure()
ax = plt.subplot()
ax.set_xlabel('Superposn ms=0 frac.')
ax.set_ylabel('Ebit rate')
plt.errorbar(super_posn_frac,ebit_rate_m,ebit_rate_m_u,label = '-')
plt.errorbar(super_posn_frac,ebit_rate_p,ebit_rate_p_u,label = '+')
plt.legend()
plt.show()
plt.close('all')

fig  = plt.figure()
ax = plt.subplot()
ax.set_xlabel('Superposn ms=0 frac.')
ax.set_ylabel('Raw rate (Hz)')
plt.plot(super_posn_frac,rate_m,label = '-')
plt.plot(super_posn_frac,rate_p,label = '+')
plt.legend()
plt.show()
plt.close('all')

fig  = plt.figure()
ax = plt.subplot()
ax.set_xlabel('Superposn ms=0 frac.')
ax.set_ylabel('Avg fidelity')
plt.errorbar(super_posn_frac,avg_fidelity_m,avg_fidelity_m_u,label = '-')
plt.errorbar(super_posn_frac,avg_fidelity_p,avg_fidelity_p_u,label = '+')
plt.legend()
plt.show()
plt.close('all')